In [1]:
import pandas as pd
import numpy as np

print('Версия pandas:', pd.__version__)

Версия pandas: 2.1.4


# Представление данных при помощи ограниченных интервалов pandas

# Объекты Interval, IntervalArray, IntervalIndex

# Часть 1 - Примеры представления данных в интервалах

### Функция для разделения данных на интервалы 
### pd.cut(x, bins, right=True, labels=None...)
    x - набор данных для разделения на интервалы
    bins - точки-границы интервалов 
    (может быть одним числом, тогда обозначенное количество интервалов будет иметь одинаковую длину)
    right=True - по умолчанию правый конец диапазона включен 
    labels - список с названиями интервалов (можно вообще не указывать, по умолчанию labels=None)

In [2]:
# Снова откроем файл 'people' с информацией о 10 людях
df=pd.read_csv('files/people', usecols=['Имя','Возраст'])
df

,Имя,Возраст
0,Гедеон Леви,14
1,Патрик Бреннан,18
2,Захар Овчинников,70
3,Нкиру Нзегву,45
4,Амели Лажуа,26
5,Хавьер Мартинес,8
6,Том Дэвис,40
7,Бакир Кхалед,58
8,Джи Чанг,21
9,Ахана Джа,89


In [3]:
# создадим список с точками-границами возрастных диапазонов
# этот список мы передадим параметром bins
age_points=[0, 12, 18, 35, 50, 65, 80, 100]

In [4]:
# передаем набор для разделения df['Возраст']
# передаем точки для разделения age_points
# по умолчанию правый конец диапазона включен. но нам нужно включить левый конец, поэтому указываем right=False
result1 = pd.cut(x=df['Возраст'], bins=age_points, right=False)
result1

0     [12, 18)
1     [18, 35)
2     [65, 80)
3     [35, 50)
4     [18, 35)
5      [0, 12)
6     [35, 50)
7     [50, 65)
8     [18, 35)
9    [80, 100)
Name: Возраст, dtype: category
Categories (7, interval[int64, left]): [[0, 12) < [12, 18) < [18, 35) < [35, 50) < [50, 65) < [65, 80) < [80, 100)]

In [5]:
# проделываем все то же самое, только в этот раз даем названия интервалам, передав их списком в labels
result2=pd.cut(df['Возраст'],
       bins=age_points,
       right=False,
       labels=['дети','подростки','молодежь','средний возраст','зрелый возраст','пожилой возраст','старость'])
result2

0          подростки
1           молодежь
2    пожилой возраст
3    средний возраст
4           молодежь
5               дети
6    средний возраст
7     зрелый возраст
8           молодежь
9           старость
Name: Возраст, dtype: category
Categories (7, object): ['дети' < 'подростки' < 'молодежь' < 'средний возраст' < 'зрелый возраст' < 'пожилой возраст' < 'старость']

In [6]:
# создадим две новые колонки с двумя результатами разбиения возрастов на группы
df['Возрастные интервалы']=result1
df['Возрастная категория']=result2
df

,Имя,Возраст,Возрастные интервалы,Возрастная категория
0,Гедеон Леви,14,"[12, 18)",подростки
1,Патрик Бреннан,18,"[18, 35)",молодежь
2,Захар Овчинников,70,"[65, 80)",пожилой возраст
3,Нкиру Нзегву,45,"[35, 50)",средний возраст
4,Амели Лажуа,26,"[18, 35)",молодежь
5,Хавьер Мартинес,8,"[0, 12)",дети
6,Том Дэвис,40,"[35, 50)",средний возраст
7,Бакир Кхалед,58,"[50, 65)",зрелый возраст
8,Джи Чанг,21,"[18, 35)",молодежь
9,Ахана Джа,89,"[80, 100)",старость


#### Каждое отдельное значение колонки 'Возрастные интервалы' - это объект Interval

In [7]:
# каждое значение колонки с интервалами 'Возрастные интервалы'- это объект Interval
print(df['Возрастные интервалы'][0])
print(type(df['Возрастные интервалы'][0]))
df['Возрастные интервалы'][0]

[12, 18)
<class 'pandas._libs.interval.Interval'>


Interval(12, 18, closed='left')

## Разбиваем данные на интервальные категории без функции cut
#### Используем IntervalArray.from_breaks + pd.Categorical

In [8]:
df2=pd.read_csv('files/people', usecols=['Имя','Возраст'])
print(age_points)
df2

[0, 12, 18, 35, 50, 65, 80, 100]


,Имя,Возраст
0,Гедеон Леви,14
1,Патрик Бреннан,18
2,Захар Овчинников,70
3,Нкиру Нзегву,45
4,Амели Лажуа,26
5,Хавьер Мартинес,8
6,Том Дэвис,40
7,Бакир Кхалед,58
8,Джи Чанг,21
9,Ахана Джа,89


#### Создаем массив интервалов IntervalArray с помощью pd.arrays.IntervalArray.from_breaks

In [9]:
# создаем интервальный массив IntervalArray, используя точки данных из переменной age_points
# указываем параметр closed='left', чтобы закрыть левый конец каждого интервала (по умолчанию закрывается правый конец)
i_arr=pd.arrays.IntervalArray.from_breaks(age_points, closed='left')
i_arr

<IntervalArray>
[[0, 12), [12, 18), [18, 35), [35, 50), [50, 65), [65, 80), [80, 100)]
Length: 7, dtype: interval[int64, left]

In [10]:
# создаем объект Categorical с интервалами-категориями 
cat_i_arr=pd.Categorical(i_arr)
cat_i_arr

[[0, 12), [12, 18), [18, 35), [35, 50), [50, 65), [65, 80), [80, 100)]
Categories (7, interval[int64, left]): [[0, 12), [12, 18), [18, 35), [35, 50), [50, 65), [65, 80), [80, 100)]

In [11]:
# создаем категориальный массив на основе колонки 'Возраст'
pd.Categorical(df2['Возраст'], categories=cat_i_arr)

[[12, 18), [18, 35), [65, 80), [35, 50), [18, 35), [0, 12), [35, 50), [50, 65), [18, 35), [80, 100)]
Categories (7, interval[int64, left]): [[0, 12), [12, 18), [18, 35), [35, 50), [50, 65), [65, 80), [80, 100)]

In [12]:
# присваиваем результат новой колонке df2['Интервалы']
df2['Интервалы']=pd.Categorical(df2['Возраст'], categories=cat_i_arr)
df2

,Имя,Возраст,Интервалы
0,Гедеон Леви,14,"[12, 18)"
1,Патрик Бреннан,18,"[18, 35)"
2,Захар Овчинников,70,"[65, 80)"
3,Нкиру Нзегву,45,"[35, 50)"
4,Амели Лажуа,26,"[18, 35)"
5,Хавьер Мартинес,8,"[0, 12)"
6,Том Дэвис,40,"[35, 50)"
7,Бакир Кхалед,58,"[50, 65)"
8,Джи Чанг,21,"[18, 35)"
9,Ахана Джа,89,"[80, 100)"


#### Создаем IntervalIndex

In [13]:
df2['Интервалы'].to_list()

[Interval(12, 18, closed='left'),
 Interval(18, 35, closed='left'),
 Interval(65, 80, closed='left'),
 Interval(35, 50, closed='left'),
 Interval(18, 35, closed='left'),
 Interval(0, 12, closed='left'),
 Interval(35, 50, closed='left'),
 Interval(50, 65, closed='left'),
 Interval(18, 35, closed='left'),
 Interval(80, 100, closed='left')]

In [14]:
df3=pd.read_csv('files/people', usecols=['Имя','Возраст'])
display(df3)
df3.index=df2['Интервалы'].to_list()
df3.index.name='Index - Интервалы'
display(df3)
print(type(df3.index))

,Имя,Возраст
0,Гедеон Леви,14
1,Патрик Бреннан,18
2,Захар Овчинников,70
3,Нкиру Нзегву,45
4,Амели Лажуа,26
5,Хавьер Мартинес,8
6,Том Дэвис,40
7,Бакир Кхалед,58
8,Джи Чанг,21
9,Ахана Джа,89


,Имя,Возраст
Index - Интервалы,,
"[12, 18)",Гедеон Леви,14
"[18, 35)",Патрик Бреннан,18
"[65, 80)",Захар Овчинников,70
"[35, 50)",Нкиру Нзегву,45
"[18, 35)",Амели Лажуа,26
"[0, 12)",Хавьер Мартинес,8
"[35, 50)",Том Дэвис,40
"[50, 65)",Бакир Кхалед,58
"[18, 35)",Джи Чанг,21


<class 'pandas.core.indexes.interval.IntervalIndex'>


In [15]:
df3.loc[20]

,Имя,Возраст
Index - Интервалы,,
"[18, 35)",Патрик Бреннан,18
"[18, 35)",Амели Лажуа,26
"[18, 35)",Джи Чанг,21


#### Если выбор данных будет осуществляется с использованием класса Interval(), то нам вернутся только точные совпадения

In [16]:
# запрос всех записей существующего интервала
df3.loc[pd.Interval(35, 50, closed='left')]
# а такая запись выдаст ошибку KeyError, поскольку такого интервала нет: df3.loc[pd.Interval(20, 25, closed='left')]

,Имя,Возраст
Index - Интервалы,,
"[35, 50)",Нкиру Нзегву,45
"[35, 50)",Том Дэвис,40


    для создания объекта IntervalIndex мы можем использовать функцию pandas.interval_range